In [51]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [52]:
import pprint
import pandas as pd
import ast
import os
from getpass import getpass
from openai import OpenAI

lfud_df_original = pd.read_csv('https://raw.githubusercontent.com/YandaGo/LFUD/main/LFUD.csv')[['proposition', 'sentence', 'fallacy_type', 'task2']]

# Get only the first sentence of the question from task2 and set it as the context column
lfud_df_original["context"] = lfud_df_original["task2"].apply(lambda x: ast.literal_eval(x)["question"].split('\n')[0])

# Drop task2 column and rename the columns
lfud_df = lfud_df_original.drop(columns = ["task2"]).rename(columns = {
    "fallacy_type" : "output",
    "sentence": "input"
})

# Join all unique contexts to a full context string to use for prompt
# This will contain definitions for each of the 12 LF types
full_context = ' \n'.join(lfud_df['context'].unique())

lfud_df.drop(columns = 'context', inplace = True)
display(lfud_df)

,proposition,input,output
0,All electronic products need electricity.,All electronic products need electricity. Elec...,faulty generalization
1,All electronic products need electricity.,Since all electronic products need electricity...,false causality
2,All electronic products need electricity.,All electronic products function because they ...,circular reasoning
3,All electronic products need electricity.,Most people think that all electronic products...,ad populum
4,All electronic products need electricity.,"""Either every electronic item operates using e...",false dilemma
...,...,...,...
799,It may rain tomorrow.,It might rain tomorrow. Think about how disapp...,appeal to emotion
800,It may rain tomorrow.,Person X: It may rain tomorrow. Person Y: So y...,fallacy of extension
801,It may rain tomorrow.,She knows everything and he said it may rain t...,fallacy of credibility
802,It may rain tomorrow.,If you can't prove that it won't rain tomorrow...,intentional fallacy


# Create dictionary of fallacy types and examples


In [77]:
lfud_df['output'].unique()

array(['faulty generalization', 'false causality', 'circular reasoning',
       'ad populum', 'false dilemma', 'fallacy of relevance',
       'ad hominem', 'appeal to emotion', 'fallacy of extension',
       'fallacy of credibility', 'intentional fallacy',
       'deductive fallacy'], dtype=object)

In [53]:
fallacy_definitions_and_examples = {}
for index, row in lfud_df_original[['fallacy_type', 'context']].drop_duplicates().iterrows():
    fallacy_definitions_and_examples[row['fallacy_type']] = {'Definition': row['context']}

for fallacy in fallacy_definitions_and_examples:
    fallacy_examples_df = lfud_df_original[lambda df: df['fallacy_type'] == fallacy].head(3)
    fallacy_definitions_and_examples[fallacy]['Examples'] = {
        "Proposition 1": fallacy_examples_df['proposition'].iloc[0],
        "Result 1": fallacy_examples_df['sentence'].iloc[0],

        "Proposition 2": fallacy_examples_df['proposition'].iloc[1],
        "Result 2": fallacy_examples_df['sentence'].iloc[1],

        "Proposition 3": fallacy_examples_df['proposition'].iloc[2],
        "Result 3": fallacy_examples_df['sentence'].iloc[2],

    }

pprint.pp(fallacy_definitions_and_examples['faulty generalization'])

{'Definition': 'Faulty generalization occurs when a conclusion about all or '
               'many instances of a phenomenon is drawn from one or a few '
               'instances of that phenomenon.',
 'Examples': {'Proposition 1': 'All electronic products need electricity.',
              'Result 1': 'All electronic products need electricity. '
                          'Electronics are part of my family. Therefore, '
                          'everything in my house needs electricity.',
              'Proposition 2': "All flowers don't stay open forever.",
              'Result 2': "All flowers don't stay open forever. Roses are a "
                          'type of plants. Therefore, all plants do not stay '
                          'open forever.',
              'Proposition 3': 'Everyone in my family has never been to '
                               'Europe.',
              'Result 3': 'Everyone in my family has never been to Europe. My '
                          'family is a

# Create prompt prefixes for each fallacy type to generate more data for fine-tuning

In [67]:
fallacy_prompt_instructions = {k: '' for k in fallacy_definitions_and_examples}

for fallacy in fallacy_definitions_and_examples:
    definition = fallacy_definitions_and_examples[fallacy]['Definition']
    examples = fallacy_definitions_and_examples[fallacy]['Examples']
    prompt_instruction = (
        f"1. Generate the sentence with {fallacy}. {definition}\n"
        "2. The sentence should have complete premise and conclusion, but try not to make it too long.\n"
        "3. Start off each generated output with the type of fallacy you are generating.\n"

        f"Proposition 1: {examples['Proposition 1']}\nResult 1: {examples['Result 1']}\n\n"
        f"Proposition 2: {examples['Proposition 2']}\nResult 2: {examples['Result 2']}\n\n"
        f"Proposition 3: {examples['Proposition 3']}\nResult 3: {examples['Result 3']}\n\n"
    )

    fallacy_prompt_instructions[fallacy] = prompt_instruction

# Display the prompt to use to generate examples for faulty generalization fallacy
pprint.pp(fallacy_prompt_instructions['faulty generalization'])

('1. Generate the sentence with faulty generalization. Faulty generalization '
 'occurs when a conclusion about all or many instances of a phenomenon is '
 'drawn from one or a few instances of that phenomenon.\n'
 '2. The sentence should have complete premise and conclusion, but try not to '
 'make it too long.\n'
 '3. Start off each generated output with the type of fallacy you are '
 'generating.\n'
 'Proposition 1: All electronic products need electricity.\n'
 'Result 1: All electronic products need electricity. Electronics are part of '
 'my family. Therefore, everything in my house needs electricity.\n'
 '\n'
 "Proposition 2: All flowers don't stay open forever.\n"
 "Result 2: All flowers don't stay open forever. Roses are a type of plants. "
 'Therefore, all plants do not stay open forever.\n'
 '\n'
 'Proposition 3: Everyone in my family has never been to Europe.\n'
 'Result 3: Everyone in my family has never been to Europe. My family is a '
 'group of Americans, therefore, all 

# List of propositions from "An Introduction to Logic (14th Edition) (Hurley)" and generated from GPT tailored towards the LFUD paper’s selection criteria (clarity, evaluability, neutrality, and logical simplicity)

In [ ]:
fallacy_of_extension = [
    "Chocolate truffles are loaded with calories, so eating one is basically the same as eating a bucket of lard.",
    "Melatonin helps with jet lag, so clearly it's a miracle cure for all sleep problems.",
    ...
    "Some people believe in aliens, so we’re clearly being watched and manipulated."
]


In [79]:
propositions = [
  "Chocolate truffles are loaded with calories.",
  "Melatonin helps relieve jet lag.",

  "Not all political candidates are completely truthful." ,
  "Some spouses have been unfaithful in relationships.",
  "Naomi Osaka plays professional tennis.",

  "All film stars are celebrities.",
  "Some film stars are men.",

  "Tortured prisoners will say anything just to relieve the pain.",

  "All mammals have hearts.",
  "Some books are written by artificial intelligence.",
  "Every Olympic athlete trains daily.",

  "Some teachers assign homework over the weekend.",
  "My neighbor has never traveled abroad.",
  "Most children enjoy playing outside.",
  "Jon has cooked steak that other chefs consider restaurant quality."

  "Gravity pulls objects toward Earth.",
  "Cryptocurrency prices fluctuate wildly.",
  "The sun sets in the west.",
  "Honey never spoils.",
  "Many people believe time helps emotional recovery.",

  "The Great Wall of China is visible from space.",
  "Some lawyers have been accused of dishonesty in court.",
  "Most social media platforms are addictive.",
  "Nothing can travel faster than light.",
  "Nobody enjoys waiting in traffic.",

  "Some dogs are trained to detect seizures.",
  "Not all electric cars are environmentally friendly.",
  "Albert Einstein developed the theory of relativity.",
  "Every continent has experienced climate change.",
  "Bananas contain potassium.",
  "The internet was invented before smartphones.",
  "No reptiles produce milk.",
  "Most modern airplanes can fly above 30,000 feet.",
  "Some people believe in extraterrestrial life."
]

In [56]:
fallacy_prompt_instructions.keys()
print(fallacy_prompt_instructions['faulty generalization'])

1. Generate the sentence with faulty generalization. Faulty generalization occurs when a conclusion about all or many instances of a phenomenon is drawn from one or a few instances of that phenomenon.
2. The sentence should have complete premise and conclusion, but try not to make it too long.
3. Start off each generated output with the type of fallacy you are generating.Proposition 1: All electronic products need electricity.
Result 1: All electronic products need electricity. Electronics are part of my family. Therefore, everything in my house needs electricity.

Proposition 2: All flowers don't stay open forever.
Result 2: All flowers don't stay open forever. Roses are a type of plants. Therefore, all plants do not stay open forever.

Proposition 3: Everyone in my family has never been to Europe.
Result 3: Everyone in my family has never been to Europe. My family is a group of Americans, therefore, all Americans have never been to Europe.




## Generate the main prompt consisting of propositions and results for GPT to fill in

In [57]:
main_prompt = ""
for i in range(len(propositions)):
  main_prompt += (
      f"Proposition {i+4}: {propositions[i]}\n"
      f"Result {i+4}:\n\n"
  )

# Combine prompt prefixes with main_prompt and then pass to GPT

In [58]:
fallacy_prompts = {}
for fallacy in fallacy_prompt_instructions:
  fallacy_prompts[fallacy] = fallacy_prompt_instructions[fallacy] + main_prompt

# Save GPT output to text file

In [69]:
system_prompt = (
    "As a logician, when presented with a proposition, your objective is to simulate the way of human thinking, "
    "generating a sentence with specific type of logical fallacy. The generation should follow these instructions:\n"
)

for fallacy in fallacy_prompts:
  full_prompt = system_prompt + fallacy_prompts[fallacy]

  with open('gdrive/MyDrive/w266/Final Project/data/gpt_fallacy_prompts.txt', 'a') as f:
    f.write(full_prompt + '\n\n')

In [ ]:
from openai import OpenAI

path_to_file_name = 'gdrive/MyDrive/w266/Final Project/data/gpt_fallacy_output.txt'

system_prompt = (
    "As a logician, when presented with a proposition, your objective is to simulate the way of human thinking, "
    "generating a sentence with specific type of logical fallacy. The generation should follow these instructions:"
)

# Prompt user to enter the key securely
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ").strip()

# Replace "YOUR_API_KEY" with your actual API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


for fallacy in fallacy_prompts:
  # Construct the Prompt Message
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": fallacy_prompts[fallacy]}
  ]

  # Make the API Call
  response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
  )

  ai_response_content = response.choices[0].message.content

  with open(path_to_file_name, 'a') as f:
    f.write(ai_response_content + '\n\n')


# Transform GPT output (Python lists of fallacies for each fallacy type) into dataframe following LFUD schema ("proposition, "input", "output")

In [81]:
faulty_generalization = [
    "Chocolate truffles are loaded with calories. I once ate a chocolate bar that was high in calories. Therefore, all chocolate products must be unhealthy.",
    "Melatonin helps relieve jet lag. My friend used it once and felt better. Therefore, melatonin cures all sleep problems.",
    "Not all political candidates are completely truthful. One candidate lied during a debate. Therefore, no politician can be trusted.",
    "Some spouses have been unfaithful in relationships. My neighbor’s spouse cheated. Therefore, no spouse is ever faithful.",
    "Naomi Osaka plays professional tennis. I saw her win a match. Therefore, all tennis players must be champions like her.",
    "All film stars are celebrities. I saw one on TV last night. Therefore, anyone who appears on TV must be a celebrity.",
    "Some film stars are men. Brad Pitt is a male film star. Therefore, most film stars must be men.",
    "Tortured prisoners will say anything just to relieve the pain. One prisoner confessed falsely under torture. Therefore, no confession under pressure is ever true.",
    "All mammals have hearts. I have a dog that’s a mammal. Therefore, every animal must have a heart.",
    "Some books are written by artificial intelligence. I read one that was. Therefore, soon all books will be written by AI.",
    "Every Olympic athlete trains daily. My cousin trains every day. Therefore, she must be an Olympic athlete.",
    "Some teachers assign homework over the weekend. My teacher did last week. Therefore, all teachers give weekend homework.",
    "My neighbor has never traveled abroad. He's a retired man. Therefore, all retired people must have stayed in their home country.",
    "Most children enjoy playing outside. My daughter loves the park. Therefore, every child must love being outdoors.",
    "Jon has cooked steak that other chefs consider restaurant quality. I tried it once. Therefore, Jon is better than most chefs.",
    "Cryptocurrency prices fluctuate wildly. I lost money on Bitcoin once. Therefore, all cryptocurrencies are a scam.",
    "The sun sets in the west. I saw it set there yesterday. Therefore, it must set in the exact same spot every day.",
    "Honey never spoils. I’ve had the same jar for years. Therefore, all sweet foods last forever.",
    "Many people believe time helps emotional recovery. My cousin got over a breakup in a week. Therefore, time heals everything.",
    "The Great Wall of China is visible from space. My teacher said so once. Therefore, all large buildings can be seen from space.",
    "Some lawyers have been accused of dishonesty in court. One lawyer lied in a case I followed. Therefore, all lawyers are dishonest.",
    "Most social media platforms are addictive. My friend checks Instagram constantly. Therefore, all apps are addictive.",
    "Nothing can travel faster than light. My science teacher told me that. Therefore, no future discovery will ever change this.",
    "Nobody enjoys waiting in traffic. I get annoyed every time. Therefore, traffic should be banned entirely.",
    "Some dogs are trained to detect seizures. I saw one in a video. Therefore, all dogs are capable of being service animals.",
    "Not all electric cars are environmentally friendly. I read about one model with issues. Therefore, electric cars are no better than gas ones.",
    "Albert Einstein developed the theory of relativity. He was a physicist. Therefore, all physicists create groundbreaking theories.",
    "Every continent has experienced climate change. It snowed last week in my city. Therefore, climate change must cause every weather event.",
    "Bananas contain potassium. I eat them after workouts. Therefore, bananas are the best health food.",
    "The internet was invented before smartphones. I remember using dial-up. Therefore, smartphones are just a minor upgrade to the internet.",
    "No reptiles produce milk. I’ve never seen a lizard near milk. Therefore, reptiles can't care for their young.",
    "Most modern airplanes can fly above 30,000 feet. I flew on one last summer. Therefore, any machine that flies must reach that height.",
    "Some people believe in extraterrestrial life. My friend saw a UFO. Therefore, aliens definitely exist and visit Earth."
]

appeal_to_emotion = [
    "Chocolate truffles are loaded with calories. Just imagine how ashamed you'll feel if your clothes stop fitting.",
    "Melatonin helps relieve jet lag. Think of how miserable and exhausted you'll be if you don’t take it.",
    "Not all political candidates are truthful. Don’t you feel betrayed just thinking about how they lie to us?",
    "Some spouses have been unfaithful. Don’t you feel heartbroken thinking about how marriage can be ruined like that?",
    "Naomi Osaka plays professional tennis. Don’t you want to support her? She’s gone through so much.",
    "All film stars are celebrities. Imagine how crushed they’d be if people stopped caring about them.",
    "Some film stars are men. Doesn’t it make you angry that we even have to mention that?",
    "Tortured prisoners will say anything. Imagine the unbearable pain they must be going through.",
    "All mammals have hearts. Isn’t it beautiful how we all share something so essential?",
    "Some books are written by AI. Doesn’t it scare you that machines are replacing human creativity?",
    "Every Olympic athlete trains daily. Don’t you feel lazy now?",
    "Some teachers assign weekend homework. Isn’t it cruel to ruin a child’s weekend like that?",
    "My neighbor has never traveled abroad. Don’t you feel sorry for someone who’s never seen the world?",
    "Most children enjoy playing outside. Wouldn’t it be tragic to deny them that joy?",
    "Jon has cooked restaurant-quality steak. Wouldn’t it be rude not to compliment him?",
    "Cryptocurrency prices fluctuate wildly. Imagine how terrified investors must be.",
    "The sun sets in the west. Isn’t it poetic how nature just knows what to do?",
    "Honey never spoils. Isn’t it comforting to know that some things last forever?",
    "Many believe time heals emotional wounds. Don’t you want to believe that pain doesn’t last forever?",
    "The Great Wall of China is visible from space. Don’t you feel amazed by how incredible humans can be?",
    "Some lawyers have been dishonest. Don’t you feel angry that justice can be corrupted?",
    "Most social media platforms are addictive. Think about the poor kids wasting their lives.",
    "Nothing can travel faster than light. Doesn’t that make you feel insignificant?",
    "Nobody enjoys waiting in traffic. Isn’t it frustrating just thinking about it?",
    "Some dogs detect seizures. Wouldn’t you feel safer with one around?",
    "Not all electric cars are environmentally friendly. Doesn’t it make you feel lied to by green companies?",
    "Albert Einstein developed relativity. Don’t you admire his genius and feel humbled?",
    "Every continent has experienced climate change. Don’t you worry about the future of your children?",
    "Bananas contain potassium. Wouldn’t it be tragic if people didn’t know that and got sick?",
    "The internet was invented before smartphones. Doesn’t it feel like everything is moving too fast?",
    "No reptiles produce milk. Isn’t it weird and unsettling how different they are?",
    "Most modern airplanes can fly above 30,000 feet. Don’t you feel uneasy flying that high?",
    "Some people believe in extraterrestrial life. Isn’t that exciting—and just a little bit scary?"
]

fallacy_of_extension = [
    "Chocolate truffles are loaded with calories, so eating one is basically the same as eating a bucket of lard.",
    "Melatonin helps with jet lag, so clearly it's a miracle cure for all sleep problems.",
    "Some political candidates are dishonest, so politics is just legalized lying.",
    "Some spouses have been unfaithful, so marriage is just a failed institution.",
    "Naomi Osaka plays tennis, so anyone who likes her must be obsessed with sports.",
    "All film stars are celebrities, so clearly everyone in Hollywood is self-absorbed and fake.",
    "Some film stars are men, so obviously men are taking over the entire industry.",
    "Tortured prisoners will say anything, so every confession ever made under pressure is a complete lie.",
    "All mammals have hearts, so if something doesn’t have a heart, it can’t be alive.",
    "Some books are written by AI, so soon there’ll be no reason to read anything at all.",
    "Olympic athletes train daily, so if you don’t, you must not care about your health at all.",
    "Some teachers give weekend homework, so clearly they want students to have no life.",
    "My neighbor hasn’t traveled abroad, so he must be ignorant of everything beyond his town.",
    "Most children enjoy playing outside, so indoor kids are clearly unhealthy and antisocial.",
    "Jon has cooked steak like a chef, so he should probably open a restaurant.",
    "Crypto prices fluctuate wildly, so investing in crypto is like setting your money on fire.",
    "The sun sets in the west, so the east must be completely dark all the time.",
    "Honey never spoils, so you might as well live on honey forever.",
    "Time heals emotional wounds, so there's no need for therapy or support.",
    "The Great Wall is visible from space, so you could probably see your house from the moon.",
    "Some lawyers are dishonest, so you can’t trust any legal advice.",
    "Most social media is addictive, so anyone with a phone is a mindless zombie.",
    "Nothing travels faster than light, so faster-than-light fiction is totally worthless.",
    "Nobody likes traffic, so we should ban cars altogether.",
    "Some dogs detect seizures, so all dogs should be trained as medical professionals.",
    "Not all electric cars are green, so green tech is just one big scam.",
    "Einstein developed relativity, so he must have known everything about everything.",
    "Every continent has experienced climate change, so there’s nothing left to save.",
    "Bananas contain potassium, so if you’re low on energy, just eat bananas nonstop.",
    "The internet came before smartphones, so smartphones are just a useless add-on.",
    "No reptiles produce milk, so they’re nothing like “real” animals.",
    "Most planes fly above 30,000 feet, so flying lower must be incredibly unsafe.",
    "Some people believe in aliens, so we’re clearly being watched and manipulated."
]



circular_reasoning = [
    "Chocolate truffles are bad for you because they are unhealthy.",
    "Melatonin helps you sleep because it is a sleep aid.",
    "Not all political candidates are truthful because they often lie.",
    "Some spouses have been unfaithful because they’ve cheated.",
    "Naomi Osaka is a great player because she plays professional tennis.",
    "All film stars are celebrities because they are famous actors.",
    "Some film stars are men because some male actors are film stars.",
    "Tortured prisoners will say anything because they are being tortured.",
    "All mammals have hearts because mammals are the kind of animals that have hearts.",
    "Some books are written by AI because AI has written some books.",
    "Every Olympic athlete trains daily because daily training is what Olympic athletes do.",
    "Some teachers assign homework over the weekend because those teachers give assignments on weekends.",
    "My neighbor hasn’t traveled abroad because he’s never left the country.",
    "Most children enjoy playing outside because they like being outdoors.",
    "Jon has cooked restaurant-quality steak because it tastes like steak from a restaurant.",
    "Cryptocurrency prices fluctuate because they change a lot.",
    "The sun sets in the west because that’s where the sun goes down.",
    "Honey never spoils because it doesn’t go bad.",
    "Time heals emotional wounds because it helps people recover emotionally over time.",
    "The Great Wall is visible from space because astronauts can see it from space.",
    "Some lawyers are dishonest because they don’t always tell the truth.",
    "Most social media is addictive because people get addicted to it.",
    "Nothing travels faster than light because the speed of light is the fastest.",
    "Nobody enjoys traffic because it’s unenjoyable.",
    "Some dogs detect seizures because they are seizure-detecting dogs.",
    "Not all electric cars are green because some electric cars are not environmentally friendly.",
    "Einstein developed the theory of relativity because it was developed by Einstein.",
    "Every continent has experienced climate change because climate change has affected all continents.",
    "Bananas contain potassium because potassium is in bananas.",
    "The internet was invented before smartphones because smartphones came after the internet.",
    "No reptiles produce milk because reptiles are animals that don’t make milk.",
    "Most modern airplanes can fly above 30,000 feet because that’s the altitude they usually reach.",
    "Some people believe in extraterrestrial life because they think aliens exist."
]

false_causality = [
    "Chocolate truffles are loaded with calories, so they must be the reason I’m always tired.",
    "Melatonin helps with jet lag, so jet lag must be caused by not taking melatonin.",
    "Not all political candidates are truthful, so voting must cause dishonesty.",
    "Some spouses have been unfaithful, which is probably why marriages are expensive.",
    "Naomi Osaka plays tennis, so she must have caused tennis to become popular.",
    "All film stars are celebrities, so becoming a celebrity must make you act in films.",
    "Some film stars are men, so men must be the reason movies exist.",
    "Tortured prisoners will say anything, so torture must make people intelligent.",
    "All mammals have hearts, so the heart must be what makes something a mammal.",
    "Some books are written by AI, so AI must be the reason people read more now.",
    "Olympic athletes train daily, so training daily must make you famous.",
    "Some teachers assign weekend homework, so the weekend must be why students hate school.",
    "My neighbor hasn’t traveled abroad, so travel bans must be in place.",
    "Most children enjoy playing outside, so fresh air must cause childhood.",
    "Jon’s steak tastes great, so clearly he has a five-star kitchen.",
    "Crypto fluctuates wildly, so it must be because of internet memes.",
    "The sun sets in the west, so western countries must control the sun.",
    "Honey never spoils, so it must be the reason bees live so long.",
    "Many people believe time heals emotional wounds, so clocks must be therapeutic.",
    "The Great Wall is visible from space, so China built it to be seen by aliens.",
    "Some lawyers are dishonest, so law school must teach dishonesty.",
    "Most social media is addictive, so phone screens must release chemicals.",
    "Nothing travels faster than light, so light must be slowing everything else down.",
    "Nobody enjoys traffic, so city planning must be a punishment.",
    "Some dogs detect seizures, so seizures must emit dog-attracting signals.",
    "Not all electric cars are green, so electricity must be worse than gas.",
    "Einstein developed relativity, so German education must cause genius.",
    "Every continent has climate change, so geography must cause warming.",
    "Bananas contain potassium, so eating them must improve cell phone signals.",
    "The internet came before smartphones, so the internet must have invented phones.",
    "No reptiles produce milk, so milk must repel cold-blooded animals.",
    "Most airplanes fly above 30,000 feet, so altitude must cause safety.",
    "Some people believe in aliens, so Hollywood must be in contact with them."
]

intentional_fallacies = [
    "Chocolate truffles are high in calories, and I know you care about your health—so you obviously agree they’re bad.",
    "Melatonin works for me, so clearly it’s the best solution for everyone.",
    "Not all political candidates are honest—just ask anyone who’s ever voted.",
    "Some spouses cheat, and I’ve seen enough to know that’s all we need to say.",
    "Naomi Osaka is a pro, so clearly tennis is the only sport worth watching.",
    "Film stars are celebrities, and that proves they’re better than regular people.",
    "Some film stars are men, which means gender equity in Hollywood is solved.",
    "Tortured prisoners say anything—everyone knows that, so case closed.",
    "All mammals have hearts, and you don’t need a science degree to understand that.",
    "Some books are written by AI. That’s all the proof we need that AI is taking over everything.",
    "Olympic athletes train daily, which proves daily training is the only path to success.",
    "Some teachers assign weekend homework. That’s clearly why students are stressed.",
    "My neighbor has never traveled abroad, and if that doesn’t make him uncultured, I don’t know what does.",
    "Kids enjoy playing outside—so you obviously care about their well-being, right?",
    "Jon’s steak is restaurant-quality, and saying otherwise is just jealousy.",
    "Crypto fluctuates wildly. You wouldn’t want to lose money, would you?",
    "The sun sets in the west—don’t try to overcomplicate it.",
    "Honey never spoils, and that’s all you need to know about natural foods.",
    "Time heals emotional wounds—why keep analyzing it?",
    "The Great Wall is visible from space—if you don’t believe that, you’re missing the point.",
    "Some lawyers are dishonest. That should make you rethink everything.",
    "Most social media is addictive. If you care about society, you’ll agree.",
    "Nothing’s faster than light. Stop overthinking it.",
    "Nobody enjoys traffic, so let’s stop pretending it’s not a problem.",
    "Some dogs detect seizures. That alone makes them more useful than most people.",
    "Not all electric cars are green—just admit it already.",
    "Einstein developed relativity. Argue with that and you’re denying facts.",
    "Every continent has faced climate change. So obviously, it’s real.",
    "Bananas contain potassium—do we really need a whole debate on that?",
    "The internet came before smartphones. That’s basic knowledge.",
    "No reptiles produce milk—come on, that’s not even controversial.",
    "Most airplanes fly above 30,000 feet—what else do you need to know?",
    "People believe in aliens, and if they’re wrong, why hasn’t anyone proven it?"
]

fallacy_of_credibility = [
    "Chocolate truffles are loaded with calories—just ask any celebrity nutritionist.",
    "Melatonin helps with jet lag. A famous athlete swears by it.",
    "Not all political candidates are truthful. Even my uncle who works in politics says so.",
    "Some spouses have been unfaithful. Dr. Phil says it’s a widespread problem.",
    "Naomi Osaka plays tennis. Serena Williams said she's a true pro, so she must be.",
    "All film stars are celebrities. The Academy Awards confirm their importance.",
    "Some film stars are men. An article in Men’s Health said it, so it’s true.",
    "Tortured prisoners will say anything—top intelligence agencies agree.",
    "All mammals have hearts. It’s in every biology textbook.",
    "Some books are written by AI. Elon Musk even warned about AI taking over.",
    "Every Olympic athlete trains daily. My coach told me that's how champions are made.",
    "Some teachers assign weekend homework. The PTA newsletter complained about it.",
    "My neighbor hasn’t traveled abroad. A travel blogger once said that’s a sign of cultural ignorance.",
    "Most children enjoy playing outside. Pediatricians say it’s essential for development.",
    "Jon’s steak is restaurant-quality. A food critic confirmed it.",
    "Cryptocurrency prices fluctuate. CNBC reported the same thing last night.",
    "The sun sets in the west. Every science teacher says so.",
    "Honey never spoils. The FDA even recognizes its longevity.",
    "Time heals emotional wounds. Oprah said it helped her.",
    "The Great Wall is visible from space—Neil Armstrong supposedly confirmed it.",
    "Some lawyers are dishonest. A judge once admitted it on camera.",
    "Most social media is addictive. The Surgeon General issued a warning.",
    "Nothing travels faster than light. Einstein proved it.",
    "Nobody enjoys traffic. The mayor even admitted it’s a problem.",
    "Some dogs detect seizures. A certified trainer says they’re life-saving.",
    "Not all electric cars are eco-friendly. Even Tesla has critics.",
    "Einstein developed relativity. Physicists still cite his work.",
    "Every continent has faced climate change. The UN reports confirm this.",
    "Bananas contain potassium. Nutrition labels prove it.",
    "The internet was invented before smartphones. Wikipedia says so.",
    "No reptiles produce milk. Every biology professor agrees.",
    "Most airplanes fly above 30,000 feet. The FAA recommends it.",
    "Some people believe in aliens. Even some astronauts have said so."
]

fallacy_of_relevance = [
    "Chocolate truffles are loaded with calories. But did you know the company that makes them donates to charity?",
    "Melatonin helps relieve jet lag. Anyway, it comes in a cute little bottle.",
    "Not all political candidates are truthful. But one of them has a great smile.",
    "Some spouses have been unfaithful. But love is complicated, and weddings are expensive.",
    "Naomi Osaka plays tennis. But she also wears really nice outfits on court.",
    "All film stars are celebrities. And speaking of celebrities, have you heard about their new fashion line?",
    "Some film stars are men. But the movie industry has really good popcorn.",
    "Tortured prisoners will say anything. But think about how many wars we’ve won.",
    "All mammals have hearts. Speaking of hearts, Valentine's Day is coming up.",
    "Some books are written by AI. But AI can also play chess and compose music.",
    "Every Olympic athlete trains daily. But remember, sports bring people together.",
    "Some teachers assign weekend homework. But students should be grateful for education.",
    "My neighbor has never traveled abroad. But his lawn is always neatly trimmed.",
    "Most children enjoy playing outside. But did you know playgrounds are expensive to maintain?",
    "Jon has cooked restaurant-quality steak. But he also volunteers at the animal shelter.",
    "Cryptocurrency prices fluctuate wildly. But you have to admit, the logos look cool.",
    "The sun sets in the west. But sunsets look beautiful from anywhere.",
    "Honey never spoils. Besides, it’s made by hardworking bees.",
    "Many believe time helps recovery. But calendars are often wrong, so who knows.",
    "The Great Wall is visible from space. But so are satellites, which are much smaller.",
    "Some lawyers are dishonest. But they pass the bar exam, so they must be smart.",
    "Most social media is addictive. But it helps people share birthday wishes.",
    "Nothing travels faster than light. But my Wi-Fi has been pretty fast lately.",
    "Nobody enjoys traffic. But roads look great after it rains.",
    "Some dogs detect seizures. But cats are more independent.",
    "Not all electric cars are green. But they come in fun colors.",
    "Einstein developed relativity. But he also had wild hair.",
    "Every continent has faced climate change. But penguins are still cute.",
    "Bananas contain potassium. But they’re also very easy to peel.",
    "The internet came before smartphones. But smartphones have better games.",
    "No reptiles produce milk. But they’re really good at hiding.",
    "Most airplanes fly above 30,000 feet. But the snack service is what really matters.",
    "Some people believe in extraterrestrials. But crop circles look really artistic."
]

ad_hominem = [
    "Chocolate truffles are high in calories, but only a fool would avoid them completely.",
    "Melatonin helps with jet lag, but you clearly have no idea what you're talking about if you think natural remedies work.",
    "Not all political candidates are truthful, but you’re too naive to understand how politics works.",
    "Some spouses have been unfaithful, but only bitter people bring that up.",
    "Naomi Osaka plays tennis, but people who watch tennis are just elitist snobs.",
    "All film stars are celebrities, but anyone who likes celebrities must be shallow.",
    "Some film stars are men, but coming from someone who doesn’t even watch movies, that argument means nothing.",
    "Tortured prisoners say anything, but of course you’d disagree—you’ve never been in a real conflict zone.",
    "All mammals have hearts, but that’s just basic science you probably failed in school.",
    "Some books are written by AI, but only tech-obsessed nerds care about that.",
    "Every Olympic athlete trains daily, but if you think that matters, you clearly don’t understand real discipline.",
    "Some teachers assign weekend homework, but you just hate education if you’re complaining.",
    "My neighbor has never traveled abroad, but he’s ignorant anyway, so that doesn’t surprise me.",
    "Most children enjoy playing outside, but only helicopter parents would disagree.",
    "Jon has cooked steak like a pro, but he’s always showing off anyway.",
    "Crypto fluctuates wildly, but you wouldn’t understand—it’s too advanced for someone like you.",
    "The sun sets in the west, but only a complete moron would think that matters to this argument.",
    "Honey never spoils, but only conspiracy theorists talk about food longevity like that.",
    "Time heals emotional wounds, but you wouldn’t get it—you’re emotionally stunted.",
    "The Great Wall is visible from space, but only a gullible person would believe that old myth.",
    "Some lawyers are dishonest, but you hate lawyers only because you lost a case.",
    "Most social media is addictive, but you’re just jealous because you don’t have any followers.",
    "Nothing travels faster than light, but science deniers like you wouldn’t know that.",
    "Nobody enjoys traffic, but you seem to enjoy whining more than solving problems.",
    "Some dogs detect seizures, but of course you’d argue against that—you hate animals.",
    "Not all electric cars are green, but coming from someone who drives a gas guzzler, your opinion is irrelevant.",
    "Einstein developed relativity, but you probably never even finished high school science.",
    "Every continent has experienced climate change, but climate change deniers like you don’t understand evidence.",
    "Bananas contain potassium, but you can’t even spell potassium correctly.",
    "The internet came before smartphones, but old people always cling to the past.",
    "No reptiles produce milk, but you probably think lizards are mammals.",
    "Most planes fly above 30,000 feet, but you’ve never flown in one, so how would you know?",
    "Some people believe in aliens, but people like you will believe anything on the internet."
]

false_dilemma = [
    "Chocolate truffles are loaded with calories, so you either give them up or get fat.",
    "Melatonin helps with jet lag, so you can either take it or suffer all night.",
    "Not all political candidates are truthful, so you must either distrust all of them or be a blind follower.",
    "Some spouses have been unfaithful, so either you constantly suspect your partner or you’ll be betrayed.",
    "Naomi Osaka plays tennis, so you either support her or you're against women in sports.",
    "All film stars are celebrities, so you either follow their lives or you're totally out of touch.",
    "Some film stars are men, so either you accept male dominance or you reject the film industry.",
    "Tortured prisoners will say anything, so you either believe everything is fake or trust no one ever.",
    "All mammals have hearts, so either you believe in biology or you're anti-science.",
    "Some books are written by AI, so you must choose between human writing or total automation.",
    "Every Olympic athlete trains daily, so either you train like them or you're just lazy.",
    "Some teachers assign weekend homework, so either you want smart kids or no weekends.",
    "My neighbor has never traveled abroad, so either he's totally unworldly or a shut-in.",
    "Most children enjoy playing outside, so either you let kids roam or you're stifling them.",
    "Jon has cooked steak like a pro, so either he becomes a chef or he's wasting his life.",
    "Crypto prices fluctuate wildly, so you either become rich or lose everything.",
    "The sun sets in the west, so you either believe that or deny science entirely.",
    "Honey never spoils, so either you stockpile it or risk wasting food forever.",
    "Time heals emotional wounds, so either you wait or you’ll never heal.",
    "The Great Wall is visible from space, so either you’re amazed or you're just ignorant.",
    "Some lawyers are dishonest, so either you avoid them all or get betrayed.",
    "Most social media is addictive, so you either quit or become a mindless scroll-bot.",
    "Nothing travels faster than light, so either you accept physics or believe in fairy tales.",
    "Nobody enjoys waiting in traffic, so either take public transit or stay home.",
    "Some dogs detect seizures, so either you get a dog or risk your life.",
    "Not all electric cars are green, so either drive gas or give up transportation.",
    "Einstein developed relativity, so either you believe his theories or reject all science.",
    "Every continent has faced climate change, so either we panic or ignore the planet.",
    "Bananas contain potassium, so either eat them or risk deficiency.",
    "The internet came before smartphones, so either go offline or use only old tech.",
    "No reptiles produce milk, so either they’re not animals or they’re just aliens.",
    "Most airplanes fly above 30,000 feet, so either you're safe or doomed.",
    "Some people believe in aliens, so either we’re not alone or everyone’s delusional."
]

ad_populum = [
    "Chocolate truffles must be healthy—everyone eats them at parties.",
    "Melatonin helps with jet lag—millions of people use it every night.",
    "Not all political candidates are truthful—most people already assume that.",
    "Some spouses have been unfaithful—just look at how many people get divorced.",
    "Naomi Osaka plays tennis—everyone is watching her matches.",
    "All film stars are celebrities—just ask anyone who reads magazines.",
    "Some film stars are men—most movies are led by men, so that must be the norm.",
    "Tortured prisoners will say anything—everybody knows that from the movies.",
    "All mammals have hearts—no one disputes that.",
    "Some books are written by AI—everyone’s talking about it online.",
    "Every Olympic athlete trains daily—just ask the fans.",
    "Some teachers assign weekend homework—students everywhere complain about it.",
    "My neighbor hasn’t traveled abroad—most people in this town haven’t either.",
    "Most children enjoy playing outside—just watch any playground.",
    "Jon has cooked restaurant-quality steak—everyone who's tried it says it's amazing.",
    "Crypto prices fluctuate wildly—everyone’s saying it’s the wild west of investing.",
    "The sun sets in the west—everyone learns that in school.",
    "Honey never spoils—people have believed that for centuries.",
    "Many believe time helps recovery—because that’s what everyone says.",
    "The Great Wall is visible from space—almost everyone believes that.",
    "Some lawyers are dishonest—just ask the general public.",
    "Most social media is addictive—everyone knows it ruins productivity.",
    "Nothing travels faster than light—everyone accepts that from physics class.",
    "Nobody enjoys traffic—everyone complains about it every day.",
    "Some dogs detect seizures—everyone’s seen those stories.",
    "Not all electric cars are green—most car buyers already know that.",
    "Einstein developed relativity—every student learns that in school.",
    "Every continent has experienced climate change—it's what all the news reports say.",
    "Bananas contain potassium—everyone knows bananas are good for that.",
    "The internet came before smartphones—anyone with a computer remembers that.",
    "No reptiles produce milk—everyone who took biology knows that.",
    "Most airplanes fly above 30,000 feet—just ask any frequent flyer.",
    "Some people believe in aliens—just look at all the documentaries on TV."
]

deductive_fallacies = [
    "Chocolate truffles are loaded with calories. Therefore, anything with calories must be a truffle.",
    "Melatonin helps jet lag. I’m not jet lagged, so melatonin must not help me sleep.",
    "Not all political candidates are truthful. So, if someone is truthful, they can’t be a political candidate.",
    "Some spouses have been unfaithful. Therefore, if you're faithful, you're not a spouse.",
    "Naomi Osaka plays tennis. So, everyone who plays tennis is Naomi Osaka.",
    "All film stars are celebrities. Therefore, all celebrities must be film stars.",
    "Some film stars are men. Therefore, if you're not a man, you're not a film star.",
    "Tortured prisoners will say anything. Therefore, anything said must be by a tortured prisoner.",
    "All mammals have hearts. Therefore, anything with a heart must be a mammal.",
    "Some books are written by AI. So if a book is written well, it must be AI.",
    "Every Olympic athlete trains daily. I train daily, so I must be an Olympic athlete.",
    "Some teachers assign weekend homework. So if I have no homework, I must not have a teacher.",
    "My neighbor has never traveled abroad. So if you’ve traveled, you can’t be my neighbor.",
    "Most children enjoy playing outside. Therefore, anyone who doesn’t is not a child.",
    "Jon has cooked steak like a chef. Therefore, all chefs must be named Jon.",
    "Crypto fluctuates wildly. Therefore, any fluctuating asset must be crypto.",
    "The sun sets in the west. If the sun doesn’t set in the west, it must not be the sun.",
    "Honey never spoils. This food spoiled, so it can't be honey.",
    "Many believe time heals. If something isn’t healing, time must not exist.",
    "The Great Wall is visible from space. This object isn’t, so it can’t be great.",
    "Some lawyers are dishonest. Therefore, if you're honest, you're not a lawyer.",
    "Most social media is addictive. So if it’s not addictive, it’s not social media.",
    "Nothing travels faster than light. This moved faster, so it must not exist.",
    "Nobody enjoys traffic. So if someone enjoys it, they must not be a person.",
    "Some dogs detect seizures. Therefore, if a dog can't, it must not be a real dog.",
    "Not all electric cars are green. So if it’s green, it’s not electric.",
    "Einstein developed relativity. So if someone didn’t, they’re not Einstein.",
    "Every continent has faced climate change. This place hasn’t, so it’s not a continent.",
    "Bananas contain potassium. So if it has potassium, it must be a banana.",
    "The internet came before smartphones. If something came after, it must be a smartphone.",
    "No reptiles produce milk. This animal produces milk, so it can't be a reptile.",
    "Most planes fly high. This one doesn’t, so it must not be a plane.",
    "Some people believe in aliens. So if you don’t, you’re not a person."
]

# Organize all fallacy categories into a dict
fallacy_map = {
    "faulty generalization": faulty_generalization,
    "false causality": false_causality,
    "circular reasoning": circular_reasoning,
    "ad populum": ad_populum,
    "false dilemma": false_dilemma,
    "fallacy of relevance": fallacy_of_relevance,
    "ad hominem": ad_hominem,
    "appeal to emotion": appeal_to_emotion,
    "fallacy of extension": fallacy_of_extension,
    "fallacy of credibility": fallacy_of_credibility,
    "intentional fallacy": intentional_fallacies,
    "deductive fallacy": deductive_fallacies
}

# Build the dataframe
records = []
for fallacy_type, fallacy_examples in fallacy_map.items():
    for i, fallacy_sentence in enumerate(fallacy_examples):
        if i >= len(propositions):  # safety guard in case lengths mismatch
            break
        records.append({
            "proposition": propositions[i],
            "input": fallacy_sentence,
            "output": fallacy_type
        })
path_to_csv = 'gdrive/MyDrive/w266/Final Project/data/more_lfud_data.csv'
more_lfud_df = pd.DataFrame(records)
more_lfud_df.to_csv(path_to_csv)

In [83]:
more_lfud_df = pd.DataFrame(records)
pd.concat([lfud_df, more_lfud_df])['output']

,proposition,input,output
0,All electronic products need electricity.,All electronic products need electricity. Elec...,faulty generalization
1,All electronic products need electricity.,Since all electronic products need electricity...,false causality
2,All electronic products need electricity.,All electronic products function because they ...,circular reasoning
3,All electronic products need electricity.,Most people think that all electronic products...,ad populum
4,All electronic products need electricity.,"""Either every electronic item operates using e...",false dilemma
...,...,...,...
391,Bananas contain potassium.,Bananas contain potassium. So if it has potass...,deductive fallacy
392,The internet was invented before smartphones.,The internet came before smartphones. If somet...,deductive fallacy
393,No reptiles produce milk.,No reptiles produce milk. This animal produces...,deductive fallacy
394,"Most modern airplanes can fly above 30,000 feet.","Most planes fly high. This one doesn’t, so it ...",deductive fallacy
